In [0]:
import os
import pickle
import time
import glob
import sys

# use google3 ad hoc import
#from colabtools import adhoc_import
#with adhoc_import.Google3SubmittedChangelist():
#  from google3.third_party.music21 import midi, converter, chord, note

from music21 import midi, converter, chord, note, stream, instrument, duration

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape, Bidirectional
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax, dot, concatenate
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Layer

from tensorflow.keras.utils import to_categorical, plot_model
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
!pip install essentia

     |████████████████████████████████| 11.7MB 263kB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Set Parameters and Create Folders

In [0]:
#sys.path.append('/content/gdrive/My Drive/cs230')
project_folder = "/content/gdrive/My Drive/cs230"
#data_folder = project_folder + "/data"
#jazz_folder = project_folder + "/data/sumuzhao/Jazz"
classical_folder = project_folder + "/data/sumuzhao/Classic"

In [0]:
section = 'compose'
run_id = '001'
music_name = 'Study_No_1_Opus_105'

run_folder = project_folder + '/run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.makedirs(os.path.join(run_folder, 'store'))
    os.makedirs(os.path.join(run_folder, 'output'))
    os.makedirs(os.path.join(run_folder, 'weights'))
    
mode = 'build'    # use this the on the first run
#mode = 'load'      # use this after the firts run
# data params
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

#Extract Notes

#Create Note and Duration Dictionaries

In [0]:
if mode == 'build':
    
    #music_list, parser = get_music_list(data_folder)
    #music_list = glob.glob(os.path.join(jazz_folder, "*.mid"))
    music_list = glob.glob(os.path.join(classical_folder, "-Study No.1 opus.105.mid"))
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    # These dictionaries contain the number half steps required to transpose music to either C major or A minor
    majors = dict([('A-', 4),('G#', 4),('A', 3),('A#', 2),('B-', 2),('B', 1),('C', 0),('C#', -1),('D-', -1),('D', -2),('D#', -3),('E-', -3),('E', -4),('6F', -5),('F#', 6),('G-', 6),('G', 5)])
    minors = dict([('G#', 1), ('A-', 1),('A', 0),('A#', -1),('B-', -1),('B', -2),('C', -3),('C#', -4),('D-', -4),('D', -5),('D#', 6),('E-', 6),('E', 5),('F', 4),('F#', 3),('G-', 3),('G', 2)])

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = converter.parse(file).chordify()
        
        key = original_score.analyze('key')
        #print('original key', key)
        if key.mode == "major":
            halfSteps = majors[key.tonic.name]
        elif key.mode == "minor":
            halfSteps = minors[key.tonic.name]

        score = original_score.transpose(halfSteps)
        #print('new key', score.analyze('key'))
        notes.extend(['START'] * seq_len)
        durations.extend([0]* seq_len)

        for element in score.flat:
            if isinstance(element, note.Note):
                if element.isRest:
                    notes.append(str(element.name))
                    #print(element.name)
                    durations.append(element.duration.quarterLength)
                else:
                    notes.append(str(element.nameWithOctave))
                    durations.append(element.duration.quarterLength)

            if isinstance(element, chord.Chord):
                notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) 
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f)
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

1 files in total
1 Parsing /content/gdrive/My Drive/cs230/data/sumuzhao/Classic/-Study No.1 opus.105.mid


In [0]:
notes

In [0]:
def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)

def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = {element:number for number, element in enumerate(element_names)}
    int_to_element = {number: element for number, element in enumerate(element_names)}

    return (element_to_int, int_to_element)

In [0]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

#Prepare Training Data

In [0]:
def prepare_sequences(notes, durations, lookups, distincts, seq_len=32):

    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups
    note_names, n_notes, duration_names, n_durations = distincts

    notes_network_input = []
    notes_network_output = []
    durations_network_input = []
    durations_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(len(notes) - seq_len):
        notes_sequence_in = notes[i:i + seq_len]
        notes_sequence_out = notes[i + seq_len]
        notes_network_input.append([note_to_int[char] for char in notes_sequence_in])
        notes_network_output.append(note_to_int[notes_sequence_out])

        durations_sequence_in = durations[i:i + seq_len]
        durations_sequence_out = durations[i + seq_len]
        durations_network_input.append([duration_to_int[char] for char in durations_sequence_in])
        durations_network_output.append(duration_to_int[durations_sequence_out])

    n_patterns = len(notes_network_input)

    # reshape the input into a format compatible with LSTM layers
    notes_network_input = np.reshape(notes_network_input, (n_patterns, seq_len))
    durations_network_input = np.reshape(durations_network_input, (n_patterns, seq_len))
    network_input = [notes_network_input, durations_network_input]
    notes_network_output = to_categorical(notes_network_output, num_classes=n_notes)
    durations_network_output = to_categorical(durations_network_output, num_classes=n_durations)
    network_output = [notes_network_output, durations_network_output]

    return (network_input, network_output)

In [0]:
network_input[0].shape, network_input[1].shape, network_output[0].shape, network_output[1].shape

((2100, 32), (2100, 32), (2100, 1296), (2100, 10))

In [0]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [0]:
#print(network_output[0].shape)
#print(network_output[1].shape)

In [0]:
#print('pitch input')
#print(network_input[0][5])
#print('duration input')
#print(network_input[1][5])
#print('pitch output')
#print(network_output[0][5])
#print('duration output')
#print(network_output[1][5])

# Change Log
1. add changes here

# Current Development Version of Network

In [0]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])
    x = Bidirectional(LSTM(rnn_units, return_sequences=True))(x)
    x = LSTM(rnn_units, return_sequences=True)(x)

    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)
    alpha = Activation('softmax')(e)
    alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

    c = Multiply()([x, alpha_repeated])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

    return model

In [0]:
model = create_network(n_notes, n_durations, embed_size, rnn_units)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    21200       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    600         input_4[0][0]                    
____________________________________________________________________________________________

#Training

In [0]:
weights_folder = os.path.join(run_folder, 'weights')

In [0]:
checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          #, validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )


Epoch 1/2000000
20/20 [==============================] - 1s 27ms/step - loss: 6.5795 - pitch_loss: 5.3575 - duration_loss: 1.2220
Epoch 2/2000000
20/20 [==============================] - 1s 27ms/step - loss: 6.0848 - pitch_loss: 5.1001 - duration_loss: 0.9847
Epoch 3/2000000
20/20 [==============================] - 1s 33ms/step - loss: 5.7377 - pitch_loss: 4.8413 - duration_loss: 0.8965
Epoch 4/2000000
20/20 [==============================] - 1s 26ms/step - loss: 5.4926 - pitch_loss: 4.6517 - duration_loss: 0.8409
Epoch 5/2000000
20/20 [==============================] - 1s 27ms/step - loss: 5.1999 - pitch_loss: 4.4221 - duration_loss: 0.7779
Epoch 6/2000000
20/20 [==============================] - 1s 27ms/step - loss: 4.9175 - pitch_loss: 4.2308 - duration_loss: 0.6867
Epoch 7/2000000
20/20 [==============================] - 1s 27ms/step - loss: 4.7177 - pitch_loss: 4.0915 - duration_loss: 0.6263
Epoch 8/2000000
20/20 [==============================] - 1s 27ms/step - loss: 4.5032 - pit